# The First Stars: Formation under X-ray Feedback -- Structure Visualization

This notebook generates all simulation results figures for my current paper.  All simulations run on stampede.tacc.utexas.edu

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

In [ ]:
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

In [ ]:
import pyGadget

Use the final snapshot of the vanilla simulation.

In [ ]:
sims = ['cr00', 'cr01', 'cr02', 'cr03', 'cr04', 'cr05']
n0 = [311, 327, 318, 249, 213, 176]
n5k = [1857, 1546, 1852, 1758, 1687, 1616]
t0 = '_t0'
t5k = '_t5k'
vsnap, nsnap, tag = 615, n0, t0

In [ ]:
simV = pyGadget.sim.Simulation('stampede/vanilla')
snapV = simV.load_snapshot(vsnap, track_sinks=True)

In [ ]:
snapV.gas.orient_box(centering='avg', view='face')

In [ ]:
img = pyGadget.visualize.project(snapV, 'ndensity', '5000AU', 'xy', centering='avg', depth=2.)
plt.imshow(img[2], cmap=plt.cm.Blues_r)

In [ ]:
img = pyGadget.visualize.project(snapV, 'ndensity', '5000AU', 'xz', centering='avg', depth=2.)
plt.imshow(img[2], cmap=plt.cm.Blues_r)

In [ ]:
img = pyGadget.visualize.project(snapV, 'ndensity', '5000AU', 'yz', centering='avg', depth=2.)
plt.imshow(img[2], cmap=plt.cm.Blues_r)

# 4 Panel Simulation Comparison Plots

For each sim, picking the snapshot just prior to the formation of the first sink.

In [ ]:
simlist = []
snaplist = []
for i,simname in enumerate(sims):
    sim = pyGadget.sim.Simulation('stampede/'+sims[i])
    sim.set_coordinate_system('physical')
    snap = sim.load_snapshot(nsnap[i], track_sinks=True)
    snap.gas.orient_box(centering='avg', view='face')
    simlist.append(sim)
    snaplist.append(snap)

In [ ]:
simV = pyGadget.sim.Simulation('stampede/vanilla')
sim1 = pyGadget.sim.Simulation('stampede/cr02')
sim2 = pyGadget.sim.Simulation('stampede/cr03')
sim3 = pyGadget.sim.Simulation('stampede/cr04')
snapV = simV.load_snapshot(615, track_sinks=True)#, dens_lim=1e11)
snap1 = sim1.load_snapshot(976, track_sinks=True)#, dens_lim=1e11)
snap2 = sim2.load_snapshot(350, track_sinks=True)#, dens_lim=1e11)
snap3 = sim3.load_snapshot(700, track_sinks=True)#, dens_lim=1e11)
snaplist = [snapV, snap1, snap2, snap3]

In [ ]:
for snap in snaplist:
    snap.gas.orient_box(centering='avg', view='face')

## Disk Structure

### Density structure

In [ ]:
import copy
snaplist = [snapV, snap1, snap2, snap3]
imlist = []
sinklist = []
scale = '5000AU'

#shifty = [None, None, 5000, -2000, None, None, None, None]
shifty = [None, None, None, None, None, None, None, None]

count = 0
for i,view in enumerate(['xy', 'xz']):
    for snap in snaplist:
        imlist.append(pyGadget.visualize.project(snap, 'ndensity', scale, view, centering='avg', 
                                                 depth=2., shifty=shifty[i], dens_lim=None))
        sinklist.append(copy.deepcopy(snap.sinks))
#    snap.close()

In [ ]:
from mpl_toolkits.axes_grid1 import ImageGrid

sim = ['J = 0', 'J = J$_{0}$', 'J = 10 J$_{0}$', 'J = 100 J$_{0}$']
bbox_props = dict(boxstyle="round", fc="k", ec="k", alpha=0.5)
ticks = [(7,8,9,10,11),(6,7,8,9,10)]

fig = plt.figure(1, (20, 8), dpi=600)
grid = ImageGrid(fig, 111, # similar to subplot(111)
                nrows_ncols = (2, 4), # creates 4x2 grid of axes
                axes_pad=0.0, # pad between axes in inch.
                cbar_mode = 'edge', cbar_location = 'right', cbar_size='7%', cbar_pad=0.0
                )
for i in range(8):
    x = imlist[i][0]
    y = imlist[i][1]
    im = imlist[i][2]
    ax = grid[i]
    img = ax.imshow(im, extent=[x.min(),x.max(),y.min(),y.max()], cmap=plt.cm.Blues_r, origin='lower')
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
#    img.set_clim((6.5,10.75))
    img.set_clim((8,12))
    
    cb = plt.colorbar(img, cax=grid.cbar_axes[i])
    #cb.set_ticks(ticks[i/4])
    cb.set_ticks((7,8,9,10))
    cb.ax.tick_params(left='on', labelsize=15, labelcolor='k')
    #if clabel[i]: 
    cb.set_label('Log Number Density \n    [cm$^{-3}$]')
    
    for sink in sinklist[i]:
        #mscale = sink.mass*6./27. + .33
        mscale = np.log(sink.mass) +1
        ax.plot(sink.x, sink.y, 'ko', ms=mscale, mew=1)
    ax.set_xlim(x.min(), x.max())
    ax.set_ylim(y.min(), y.max())

    if i > 3:
        cb.set_ticks(ticks[i/4])
        ax.text(0.5, 0.025, sim[i-4], color='w', ha='center', va='bottom', size=18, 
                transform=grid[i].transAxes, bbox=bbox_props)

plt.show()
fig.savefig('figures/structure/disks.png', bbox_inches='tight', dpi=100)